In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load Training Data 

In [13]:

df = pd.read_csv("~/pythonprograms/archive/twitter_training.csv", header=None)


# Clean Data 

In [15]:
def label_conversion(label):
    if label == 'Positive': 
        return 3
    elif label == 'Negative':
        return 1
    elif label == 'Neutral':
        return 2
    else: # Irrelevant
        return 0

df = df.dropna(subset=[3])
df[2] = df[2].apply(label_conversion)

/var/folders/nc/9k0fsfgj0rxg80v72klxf3900000gn/T/ipykernel_1040/11216194.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[2] = df[2].apply(label_conversion)


# Preprocess Training Data 

In [21]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df[3])
y = df[2]

# Split Training Data 

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Model 

In [26]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

# Caculate precision

In [27]:
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print('Accuracy: ', acc * 100)

Accuracy:  78.5472972972973
